# Oscilloskope utility – using Ethernet

In [27]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob
import vxi11

# Step 0:
# Connect oscilloscope via direct Ethernet link
# Step 1:
# Run "Record" on the oscilloscope
# and wait for 508 frames to be acquired.
# Step 2:
# Run this cell to initialize grabbing.


# This will need a rewrite
class TmcDriver:

    def __init__(self, device):
        print("Initializing connection to: " + device)
        self.device = device
        self.instr = vxi11.Instrument(device)
 
    def write(self, command):
        self.instr.write(command);

    def read(self, length = 500):
        return self.instr.read(length)

    def read_raw(self, length = 500):
        return self.instr.read_raw(length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
    
    def ask(self, command):
        return self.instr.ask(command)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope
        
# Default oscilloscope record timeout [s]
loop_sleep_time = 60
        
# For Ethernet
#osc = TmcDriver("TCPIP::147.231.24.72::INSTR")
osc = TmcDriver("TCPIP::192.168.13.10::INSTR")
print(osc.ask("*IDN?"))

Initializing connection to: TCPIP::192.168.13.10::INSTR
RIGOL TECHNOLOGIES,DS2102A,DS2D174501960,00.03.04.SP2


## Read repeatedly records from oscilloscope
This should be run after the initialization step. Timeout at the end should be enlarged if not all 508 frames are transferred.

In [ ]:
first_run_start_time = time.time()
raw_input()
loop_sleep_time = time.time() - first_run_start_time + 10

In [29]:
filename = 7

if (filename == 1):
    for f in glob.iglob("./data/*.h5"): # delete all .h5 files 
        print 'Deleting', f
        os.remove(f)
else:
    print 'Not removing old files, as filename {0} is not 1.'.format(filename)

while True:
    osc.write(':WAV:SOUR CHAN1')
    osc.write(':WAV:MODE NORM')
    osc.write(':WAV:FORM BYTE')
    osc.write(':WAV:POIN 1400')

    osc.write(':WAV:XINC?')
    xinc = float(osc.read(100))
    print 'XINC:', xinc,
    osc.write(':WAV:YINC?')
    yinc = float(osc.read(100))
    print 'YINC:', yinc,
    osc.write(':TRIGger:EDGe:LEVel?')
    trig = float(osc.read(100))
    print 'TRIG:', trig,
    osc.write(':WAVeform:YORigin?')
    yorig = float(osc.read(100))
    print 'YORIGIN:', yorig,
    

    osc.write(':FUNC:WREP:FEND?') # get number of last frame
    frames = int(osc.read(100))
    print 'FRAMES:', frames, 'SUBRUN', filename
    
    if (frames < 508):
        loop_sleep_time += 10

    with h5py.File('./data/data'+str(filename)+'.h5', 'w') as hf:       
        hf.create_dataset('FRAMES', data=(frames)) # write number of frames
        hf.create_dataset('XINC', data=(xinc)) # write axis parameters
        hf.create_dataset('YINC', data=(yinc))
        hf.create_dataset('TRIG', data=(trig))
        hf.create_dataset('YORIGIN', data=(yorig))
        osc.write(':FUNC:WREP:FCUR 1') # skip to n-th frame
        time.sleep(0.5)
        for n in range(1,frames+1):
            osc.write(':FUNC:WREP:FCUR ' + str(n)) # skip to n-th frame
            time.sleep(0.1)

            osc.write(':WAV:DATA?') # read data
            #time.sleep(0.4)
            wave1 = bytearray(osc.read_raw(500))
            wave2 = bytearray(osc.read_raw(500))
            wave3 = bytearray(osc.read_raw(500))
            #wave4 = bytearray(osc.read(500))
            #wave = np.concatenate((wave1[11:],wave2[:(500-489)],wave3[:(700-489)]))
            wave = np.concatenate((wave1[11:],wave2,wave3[:-1]))
            hf.create_dataset(str(n), data=wave)
    filename = filename + 1
    osc.write(':FUNC:WREC:OPER REC') # start recording
    print('  Subrun finished, sleeping for %.0f seconds.' % loop_sleep_time)
    time.sleep(loop_sleep_time) # delay for capturing

Deleting ./data/data1.h5
Deleting ./data/data2.h5
Deleting ./data/data3.h5
Deleting ./data/data4.h5
Deleting ./data/data5.h5
XINC: 5e-08 YINC: 0.0004 TRIG: 0.0074 YORIGIN: -79.0 FRAMES: 508 SUBRUN 1
  Subrun finished, sleeping for 55 seconds.
XINC: 5e-08 YINC: 0.0004 TRIG: 0.0074 YORIGIN: -79.0 FRAMES: 0 SUBRUN 2
  Subrun finished, sleeping for 65 seconds.
XINC: 5e-08 YINC: 0.0004 TRIG: 0.0074 YORIGIN: -79.0 FRAMES: 508 SUBRUN 3
  Subrun finished, sleeping for 65 seconds.
XINC: 5e-08 YINC: 0.0004 TRIG: 0.0074 YORIGIN: -79.0 FRAMES: 508 SUBRUN 4
  Subrun finished, sleeping for 65 seconds.
XINC: 5e-08 YINC: 0.0004 TRIG: 0.0074 YORIGIN: -79.0 FRAMES: 508 SUBRUN 5
  Subrun finished, sleeping for 65 seconds.
XINC: 5e-08 YINC: 0.0004 TRIG: 0.0074 YORIGIN: -79.0 FRAMES: 508 SUBRUN 6
  Subrun finished, sleeping for 65 seconds.
XINC: 5e-08 YINC: 0.0004 TRIG: 0.0074 YORIGIN: -79.0 FRAMES: 508 SUBRUN 7


KeyboardInterrupt: 